In [15]:
import json
import numpy as np

json_dir='/Users/tommy/workspace/data/psout/ezio_checker.json'

class confmat_stat(object):
    def __init__(self,path):
        try:
            with open(path, 'rb')  as f:
                self.data = json.load(f)
                #print "read data complete"
                self.enable = True
        except Exception as e:
                print e
        self.order = self.data.keys()
        self.class_good = []
        self.class_bad = []
        
    def category(self):
        for orderId in self.order:
            viewList = self.data[orderId].keys()
            for viewId in viewList:
                one_view = self.data[orderId][viewId]
                if (one_view['label'] == 'good'):
                    self.class_good.append((one_view,(orderId,viewId)))
                elif (one_view['label'] == 'bad'):
                    self.class_bad.append((one_view,(orderId,viewId)))
        print('class good total: ',len(self.class_good))
        print('class bad total: ',len(self.class_bad))
        
    def flatten_data(self,one_view):
        #print(one_view.keys())
        recall_vec = np.array(one_view['recall']).flatten()
        iou_vec = np.array(one_view['iou']).flatten()
        confmat = np.array(one_view['matrix']).flatten()
        prec_vec = np.array(one_view['precision']).flatten()
        
        feature_vec = np.array([])
        feature_vec = np.hstack((feature_vec,confmat))
        feature_vec = np.hstack((feature_vec,prec_vec))
        feature_vec = np.hstack((feature_vec,recall_vec))
        feature_vec = np.hstack((feature_vec,iou_vec))
        
        return feature_vec
    
    def concat(self):
        data = np.array([])
        label = []
        id_list = []
        dim = 0
        for class_good_, id_tuple in self.class_good:
            data = np.hstack((data,self.flatten_data(class_good_)))
            dim = len(self.flatten_data(class_good_))
            label.append(1)
            id_list.append(id_tuple)
        for class_bad_, id_tuple in self.class_bad:
            data = np.hstack((data,self.flatten_data(class_bad_)))
            label.append(2)
            id_list.append(id_tuple)
        #print(dim)
        data = np.reshape(data, (-1,dim))
        label = np.array(label)
        self.id_list = id_list
        return data,label,id_list
    
    def get_label(self,id_tuple):
        orderId,viewId = id_tuple
        one_view = self.data[orderId][viewId]
        return one_view['label']
    
stat = confmat_stat(json_dir)
stat.category()
data,label,id_list = stat.concat()
print(data.shape,label.shape,len(id_list))
print(stat.get_label(id_list[0]))

('class good total: ', 415)
('class bad total: ', 116)
((531, 18), (531,), 531)
good


In [16]:
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
X = data
y = label

print(X.shape, y.shape)
clf = LinearDiscriminantAnalysis(store_covariance=False,n_components=1)
clf.fit(X, y)
print("test acc",clf.score(X,y))

print("coef",clf.coef_)
print("interc",clf.intercept_)
print("scaling",clf.scalings_.T)

scaling = clf.scalings_.T
coef = np.array(clf.coef_)
inter = clf.intercept_

print(data[0])

for idx,test_data in enumerate(data):
    test_data = np.asmatrix(test_data)

    inference = np.dot(test_data,coef.T) + inter
    inference_prob = np.reciprocal(np.exp(-inference) + 1)
    
    raw_prediction = clf.predict(test_data)

    if(raw_prediction == 2):
        prediction = 'bad'
    elif (raw_prediction == 1):
        prediction = 'good'
    ground_truth = stat.get_label(id_list[idx])
    if(prediction != ground_truth):
        pass
        #print("class pred",prediction,"ground truth",ground_truth)
        
        #print(clf.predict_proba(test_data))
        
        #print("ID",id_list[idx])
        
    #print("transform",clf.decision_function(test_data))
#print(clf.covariance_)
#print(clf.explained_variance_ratio_)

((531, 18), (531,))
('test acc', 0.8568738229755178)
('coef', array([[ 1.06672942e-04, -1.59543073e-03,  2.04709466e-04,
         3.34274845e-03,  7.10280479e-05,  1.74612024e-03,
         3.71186253e-03, -8.92309024e-04,  1.05302839e-04,
         2.54929044e+01, -1.55655138e+01, -6.48062639e-01,
        -3.83726770e+01,  6.49433616e+01, -8.06837243e-01,
         1.03610580e+01, -3.21869585e+01,  1.54801054e+00]]))
('interc', array([-23.79531863]))
('scaling', array([[ 5.25041505e-05, -7.85266948e-04,  1.00757478e-04,
         1.64529228e-03,  3.49598244e-05,  8.59435939e-04,
         1.82696929e-03, -4.39192231e-04,  5.18297895e-05,
         1.25475427e+01, -7.66130630e+00, -3.18974782e-01,
        -1.88869340e+01,  3.19649573e+01, -3.97123238e-01,
         5.09968640e+00, -1.58423391e+01,  7.61926849e-01]]))
[2.74060000e+04 7.20000000e+01 2.50000000e+01 9.99000000e+02
 2.04270000e+04 1.63000000e+02 2.00000000e+01 8.20000000e+01
 9.82000000e+02 9.64151275e-01 9.92517370e-01 8.39316239

In [3]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
qda = QuadraticDiscriminantAnalysis()
qda.fit(X, y)
print(qda.score(X,y))

0.8079470198675497


/Users/tommy/anaconda2/lib/python2.7/site-packages/sklearn/discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
